In [1]:
# !pip install beautifulsoup4 EbookLib flet fpdf2 googletrans icrawler pandas pillow pyinstaller selenium translators undetected-chromedriver urllib3 webdriver_manager undetected-edgedriver django djangorestframework django-filter markdown dnspython mongoengine python-dotenv django-import-export django-cors-headers django-filter google_trans_new langdetect
# !pip install pymongo[snappy,gssapi,srv,tls]==3.12.3 djongo==1.3.6

In [2]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs
import translators as ts
import undetected_chromedriver as uc
import pandas as pd
from selenium import webdriver
import requests
import json
import time
import asyncio
from pymongo import MongoClient
from google_trans_new import google_translator
from langdetect import detect, DetectorFactory
from datetime import datetime
import csv
import re
DetectorFactory.seed = 0 

In [3]:
client = MongoClient('mongodb://192.168.1.11:27017/')

# Selecciona la base de datos
db = client['recopilarnovelas']

In [4]:
options = webdriver.ChromeOptions()
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3')

In [5]:
url = 'http://192.168.1.11:3000'

In [6]:
driver = webdriver.Chrome(options=options, service = Service(executable_path=ChromeDriverManager().install()))

In [7]:
driver.get(url)
# driver.maximize_window()

In [8]:
driver.find_element(By.XPATH, '/html/body/nav/a[2]').click()
time.sleep(30)

In [9]:
html = driver.page_source
soup = bs(html, 'html.parser')
div_novels = soup.find('div', class_='novelOptions')
list_novels = div_novels.find_all('a')
data_novelas = {"nombre":[], "url":[]}
for chap in list_novels:
    data_novelas['nombre'].append(chap.getText())
    data_novelas['url'].append(chap.get('href'))
df_novelas = pd.DataFrame(data_novelas)


In [10]:
len(df_novelas)

5966

In [11]:
df_novelas

,nombre,url
0,MARTIAL KING'S RETIRED LIFE,/novelas/65e7ed3d9297a4c7c5aaaeef
1,CULTIVATION CHAT GROUP,/novelas/65e874957a145405d0b24bf5
2,EMPEROR'S DOMINATION,/novelas/65efbb2ceb6e829adc4a104a
3,BECOMING IMMORTAL THROUGH GETTING MARRIED,/novelas/65efbb58eb6e829adc4a104c
4,"AS AN IMMORTAL, I ONLY LEARN FORBIDDEN SKILLS",/novelas/65f1af5d4285ef9618f6b28f
...,...,...
5961,GODZILLA IN KONOHA,/novelas/665a9909c5df59f7137311ad
5962,NARUTO: I AM NOT REALLY RELATED,/novelas/665a995fc5df59f713731567
5963,EROGE: I CAN TRAVEL TO THE ANCIENT WORLD,/novelas/665a9ac4c5df59f713732497
5964,THE LUSTFUL YOUNG MASTER IS SINISTER!,/novelas/665a9ac9c5df59f7137324b5


In [12]:
def traducir(texto):
    contenido_p = ''
    while True:
        try:
            contenido_p = ts.translate_text(
                texto, translator='bing', to_language='es')
            break
        except Exception as e:
            # print(e)
            try:
                contenido_p = ts.translate_text(
                    texto, translator='google', to_language='es')
                break
            except Exception as e:
                # print(e)
                try:
                    translator = google_translator() 
                    contenido_p = translator.translate(texto, lang_tgt='es')
                except Exception as e:
                    # print(e)
                    pass
                pass
            pass
    return contenido_p

In [13]:
def enviar_contenido_capitulo(novel_id,capitulo_id,texto_capitulo):
    url_contenido_capitulos = "http://192.168.1.11:8000/api/contenidocapitulos/"
    headers = {
        'Content-Type': 'application/json'
    }
    payload_chapter = {
        "novela_id": novel_id,
        "capitulo_id": capitulo_id,
        "texto": texto_capitulo,
        "created_at": datetime.now().isoformat(),
        "updated_at": datetime.now().isoformat()
    }

    try:
        res_cap = requests.post(url_contenido_capitulos, headers=headers, json=payload_chapter)
        res_cap.raise_for_status()
        return res_cap.json()['_id']
    except requests.exceptions.RequestException as e:
        print(f"Error al enviar el contenido del capítulo: {e}")
        return None

In [14]:
def eliminar_html(texto):
    """Elimina las etiquetas HTML del texto."""
    return re.sub(r'<[^>]+>', '', texto)

In [15]:
def manejar_driver(url, driver, novel_id):
    """
    Maneja el proceso de extracción de contenido de capítulos de una novela.

    Args:
        url (str): La URL de la novela.
        driver (webdriver): El controlador del navegador web.
        novel_id (str): El ID de la novela.

    Returns:
        None
    """
    driver.get(url)
    list_chapter = driver.find_elements(By.CLASS_NAME, 'capitulo')

    # Obtener una lista de capítulos existentes en la base de datos
    coleccion = db['app_contenidocapitulo']
    capitulos_existentes = coleccion.find({'novela_id': novel_id}, {'capitulo_id': 1, '_id': 0})
    capitulos_existentes = [doc['capitulo_id'] for doc in capitulos_existentes]

    for chapter in list_chapter:
        capitulo_id = chapter.get_attribute('id')
        if capitulo_id not in capitulos_existentes:
            print(f"Procesando capítulo: {chapter.text} id: {capitulo_id}", end=" | ")
            chapter.click()
            time.sleep(3)  # Esperar a que se cargue el contenido del capítulo

            # Cambiar el contexto al iframe
            iframe = driver.find_element(By.XPATH, "/html/body/div/div/div[1]/iframe")
            driver.switch_to.frame(iframe)

            try:
                html_frame = driver.page_source
                soup_frame = bs(html_frame, 'html.parser')
                div_chap_content = soup_frame.find('div', id='chr-content')
                p_chap_content = div_chap_content.find_all('p')
                chap_content = [p.getText().strip().rstrip() for p in p_chap_content]

                texto_capitulo = ''
                for parrafo in chap_content:
                    if parrafo != '':
                        if len(parrafo) > 4000:
                            texto_traducido = '. '.join([traducir(eliminar_html(x)) for x in parrafo.split('. ') if x != ''])
                            texto_capitulo += f"<p>{texto_traducido}</p>"
                        else:
                            texto_traducido = traducir(eliminar_html(parrafo))
                            texto_capitulo += f"<p>{texto_traducido}</p>"

                _id = enviar_contenido_capitulo(novel_id=novel_id, capitulo_id=capitulo_id, texto_capitulo=texto_capitulo)
                print(f"ID del capítulo insertado: {_id}")
            except Exception as e:
                print(f"El capitulo {capitulo_id} ha presentado problemas al momento de acceder.")
                pass

            # Volver al contexto principal de la página web
            driver.switch_to.default_content()
        # else:
        #     print(f"El capítulo {capitulo_id} ya existe en la base de datos. Omitiendo...", end=" | ")
        # break
    

In [16]:
df_novelas = df_novelas.iloc[::-1]
df_novelas

,nombre,url
5965,PROGENITOR VAMPIRE: I HAVE MANY SKILLS!,/novelas/665a9adac5df59f71373252b
5964,THE LUSTFUL YOUNG MASTER IS SINISTER!,/novelas/665a9ac9c5df59f7137324b5
5963,EROGE: I CAN TRAVEL TO THE ANCIENT WORLD,/novelas/665a9ac4c5df59f713732497
5962,NARUTO: I AM NOT REALLY RELATED,/novelas/665a995fc5df59f713731567
5961,GODZILLA IN KONOHA,/novelas/665a9909c5df59f7137311ad
...,...,...
4,"AS AN IMMORTAL, I ONLY LEARN FORBIDDEN SKILLS",/novelas/65f1af5d4285ef9618f6b28f
3,BECOMING IMMORTAL THROUGH GETTING MARRIED,/novelas/65efbb58eb6e829adc4a104c
2,EMPEROR'S DOMINATION,/novelas/65efbb2ceb6e829adc4a104a
1,CULTIVATION CHAT GROUP,/novelas/65e874957a145405d0b24bf5


In [17]:
for idx, novela in df_novelas.iterrows():
    novel_id = novela.url.split('/')[-1]
    print((idx+1),novela['nombre'],novel_id)
#     time.sleep(4)
    manejar_driver(url=f"{url}{novela.url}", driver=driver, novel_id=novel_id)
    # break

5966 PROGENITOR VAMPIRE: I HAVE MANY SKILLS! 665a9adac5df59f71373252b


In [ ]:
driver.quit()